In [1]:
from collections import OrderedDict, Counter, defaultdict
import glob 
import itertools
from itertools import izip
import os

from IPython.core.display import HTML
import gffutils
from matplotlib import gridspec
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

import clipper
from clipper.src import CLIP_analysis
from clipper.src import CLIP_analysis_display
from gscripts.general import dataviz
from gscripts import GO
from gscripts.general import region_helpers

fig_dir = "."
img_dir = "."

/nas3/gpratt/anaconda/lib/python2.7/site-packages/matplotlib/__init__.py:1318: UserWarning:  This call to matplotlib.use() has no effect
because the backend has already been chosen;
matplotlib.use() must be called *before* pylab, matplotlib.pyplot,
or matplotlib.backends is imported for the first time.

  warnings.warn(_use_error_msg)


#Set Up Data that I'll need Later

In [2]:
regions = OrderedDict()
regions['all'] = 'All'
regions["cds"] = "CDS"
regions["three_prime_utrs"] = "3' UTR"
regions["five_prime_utrs"] = "5' UTR"
regions["proxintron500"] = "Proximal\nIntron"
regions["distintron500"] = "Distal\nIntron"

assigned_regions = regions.copy()
del assigned_regions['all'] 
def move_name(interval):
    interval.name = interval[12]
    return interval

viz = CLIP_analysis_display.ClipVisualization()

def plot_denovo_motifs(bedtool, fig=None):
    root = "/nas3/gpratt/iPython_Notebook/taf15/"
    out_dir = os.path.join(root, "assigned/")
    fasta_dir = os.path.join(root, "fasta/")
    cluster_name = os.path.basename(bedtool.fn)
    cluster_out = os.path.join(root, bedtool.fn + "_homer")
                               
    cluster_regions = CLIP_analysis.assign_to_regions(tool=bedtool, 
                                                      clusters=cluster_name, 
                                                      regions=assigned_regions, 
                                                      assigned_dir = out_dir,
                                                      species="mm9"
                                                      )

    CLIP_analysis.make_fasta_files_from_regions(cluster_regions, cluster_name, fasta_dir, "/nas3/yeolab/Genome/ucsc/mm9/chromosomes/all.fa")
    CLIP_analysis.calculate_homer_motifs([5,6,7,8], regions, cluster_name, fasta_dir, cluster_out)

    if fig is None:
        fig = plt.figure(figsize=(20, 20))
        
    full_grid = gridspec.GridSpec(6, 4)
    motif_grid = gridspec.GridSpecFromSubplotSpec(1, 6,
                                                  subplot_spec=full_grid[4:6, :],
                                                  hspace=0,
                                                  wspace=0)

    viz.build_common_motifs(motif_grid, cluster_out)
    
def plot_distributions(bedtool, ax=None, label=None):
    if not ax:
        fig = plt.figure()
        ax = fig.add_subplot(1,1,1)
        
    lens = [len(interval) for interval in bedtool]
    counts = Counter(lens)
    ax.plot(counts.keys(), counts.values(), label=label)

    
def gencode_to_ensembl(gene_list):
    for gene in gene_list:
        yield gene.split(".")[0]
        
def plot_go_enrichment(df, filter_value=None, max_terms=None, **kwargs):
    df = df.copy()
    new_index = []
    for index, description in izip(df.index, df['GO Term Description']):
        new_index.append(list(index[:-1]) + [description])
    df.index = pd.MultiIndex.from_tuples(new_index)

    go_matrix = df['Bonferroni-corrected Hypergeometric p-Value'].apply(lambda x: -1 * np.log10(x))
    go_matrix = go_matrix.unstack(range(len(go_matrix.index.levels) - 1))
    go_matrix = go_matrix.fillna(0)
    
    #Set cutoff on values 
    if filter_value is not None:
        go_matrix = go_matrix[go_matrix.apply(max, axis=1) > filter_value]
    
    #Set cutoff on number of go terms to show
    if max_terms is not None:
        go_matrix = go_matrix.ix[go_matrix.max(axis=1).order(ascending=False).index].ix[:max_terms]
    sns.clustermap(go_matrix, robust=True, **kwargs)

    
gene_id_to_name = region_helpers.gene_id_to_name("/nas3/gpratt/gencode/gencode.vM1.annotation.gtf.db")
gene_id_to_type = region_helpers.gene_id_to_type("/nas3/gpratt/gencode/gencode.vM1.annotation.gtf.db")

#Old Kasey Method of getting merging

In [3]:
rpkms = glob.glob("/nas3/gpratt/projects/fet_family/analysis/fet_family_v1/*.rpkm")
rpkms = {os.path.basename(rpkm).split(".")[0]: pd.read_table(rpkm, index_col=0).RPKM for rpkm in rpkms}
rpkms = pd.concat(rpkms).unstack().T

master_df = pd.read_excel("/nas3/gpratt/Dropbox/TAF15/Data/striatum_overlap/FETT_clip_plus_halflife.xlsx", "fold_changes", index_col=0)

master_df = master_df.drop("Unnamed: 1", axis=1)

master_df['FUS UTR3 clip'][master_df['FUS UTR3 clip'] == "none"] = 0
#master_df['TAF UTR3 clip'][master_df['TAF UTR3 clip'] == "none"] = 0
master_df['TDP43 UTR3 clip'][master_df['TDP43 UTR3 clip'] == "none"] = 0
master_df = master_df.astype(float)
master_df['TAF log2 Fold change'] = master_df['TAF Fold change'].apply(np.log2)

master_df['FUS_3UTR_bound'] = master_df['FUS UTR3 clip'] > 0
master_df['TAF_3UTR_bound'] = master_df['TAF UTR3 clip'] > 0
master_df['TDP43_3UTR_bound'] = master_df['TDP43 UTR3 clip'] > 0

master_df['TDP43_FUS_3UTR_bound'] = master_df['TDP43_3UTR_bound'] & master_df['FUS_3UTR_bound']
master_df['TAF_FUS_3UTR_bound'] = master_df['TAF_3UTR_bound'] & master_df['FUS_3UTR_bound']
master_df['TDP43_TAF_3UTR_bound'] = master_df['TDP43_3UTR_bound'] & master_df['TAF_3UTR_bound']
master_df['TDP43_TAF_FUS_3UTR_bound'] = master_df['TDP43_3UTR_bound'] & master_df['TAF_3UTR_bound'] & master_df['FUS_3UTR_bound']

joined_df = rpkms.join(master_df)
joined_df.to_csv("/nas3/gpratt/Dropbox/TAF15/Data/rpkms_and_binding.csv")

rpkms.to_csv("/nas3/gpratt/Dropbox/TAF15/Data/rpkms_and_binding.csv")

/nas/nas0/gpratt/anaconda/lib/python2.7/site-packages/IPython/kernel/__main__.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/nas/nas0/gpratt/anaconda/lib/python2.7/site-packages/IPython/kernel/__main__.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


#DESeq Differental Expression Analysis / Merging

In [4]:
genes = pybedtools.BedTool("/nas3/gpratt/clipper/clipper/data/regions/mm9_genes.bed")

rbps = {"taf15": pybedtools.BedTool("/nas3/gpratt/projects/fet_family/data/stuff_for_KK/TAF15_combined_notrim_ingenes_clusters_mm950.bed"),
 "fus": pybedtools.BedTool("/nas3/gpratt/projects/fet_family/data/stuff_for_KK/TLS_hiseq_notrim_ingenes_clusters_mm950.bed"),
 "tdp43": pybedtools.BedTool("/nas3/gpratt/projects/fet_family/data/stuff_for_KK/TDP43brainclip_MP41_kcomb_notrim_ingenes_clusters_mm950.bed"),
 }



In [5]:
combined_df = pd.concat({os.path.basename(rbp).split(".")[0]: pd.read_csv(rbp, index_col=0) for rbp in glob.glob("/nas3/gpratt/Dropbox/TAF15/Data/mouse_integration/deseq/*")})
combined_df["gene_name"] = [gene_id_to_name[gene_id] for gene_id in combined_df.index.get_level_values(level=1)]
combined_df.index = pd.MultiIndex.from_tuples([rbp.split("_") + [gene] for rbp, gene in combined_df.index], names=["rbp", "cell_type", "geneid"])

ValueError: Length of names must match number of levels in MultiIndex.

#MA Plots 

In [ ]:
(27 + 12.) / (27 + 2 + 7 + 12 + 6 + 1 + 1)

In [ ]:
num_rows = 4
num_cols = 4
with dataviz.Figure(os.path.join(img_dir, "ma_plots.svg"), figsize=(num_cols * 2.5,num_rows * 2.5)) as fig:
    for x, (name, df) in enumerate(combined_df.groupby(level=["rbp", "cell_type"])):
        ax = fig.add_subplot(num_rows, num_cols, x +1)
        dataviz.ma_plot(df, ax)
        ax.set_title(" ".join(name))

#Merge in Binding to Dataframe

In [ ]:
genes = pybedtools.BedTool("/nas3/gpratt/clipper/clipper/data/regions/mm9_genes.bed")

rbps = {"taf15": pybedtools.BedTool("/nas3/gpratt/projects/fet_family/analysis/mouse_clip_v5/TAF15_WholeBrain.merged.peaks.kasey.bed"),
        'tdp43': pybedtools.BedTool("/nas3/gpratt/projects/fet_family/analysis/mouse_clip_v5/TDP43_WholeBrain.merged.peaks.kasey.bed"),
        'tls': pybedtools.BedTool("/nas3/gpratt/projects/fet_family/analysis/mouse_clip_v5/TLS_WholeBrain.merged.peaks.kasey.bed"),}


# rbps = {"taf15": pybedtools.BedTool("/nas3/gpratt/projects/fet_family/data/stuff_for_KK/TAF15_combined_notrim_ingenes_clusters_mm950.bed"),
#         "tls": pybedtools.BedTool("/nas3/gpratt/projects/fet_family/data/stuff_for_KK/TLS_hiseq_notrim_ingenes_clusters_mm950.bed"),
#         "tdp43": pybedtools.BedTool("/nas3/gpratt/projects/fet_family/data/stuff_for_KK/TDP43brainclip_MP41_kcomb_notrim_ingenes_clusters_mm950.bed"),
#  }

binding_df = []
for name, rbp in rbps.items():
    overlapping_genes = rbp.intersect(genes, wo=True, s=True).each(move_name).saveas()

    cluster_regions = CLIP_analysis.assign_to_regions(overlapping_genes, "overlapping", 
                                    assigned_regions, "assigned", nrand=0, species="mm9")
    
    for region in cluster_regions.keys():
        gene_names = pd.Series(Counter([interval.name.split(",")[0] for interval in cluster_regions[region]['real']]),
                                name=name + "_" + region)
        gene_names.name = (name, region)
        binding_df.append(gene_names)
binding_df = pd.concat(binding_df,axis=1)
binding_df.columns = pd.MultiIndex.from_tuples(binding_df.columns)
binding_df = binding_df.T
binding_df = binding_df.stack()
binding_df.index.names = ['rbp', 'region', 'gene_id']
binding_df = binding_df.sort_index()
binding_df = pd.DataFrame(binding_df)

In [ ]:
sig_df = combined_df[combined_df.padj <= .01]

In [ ]:
#THIS IS BROKEN, BUT I DON'T CARE RIGHT NOW
output = combined_df.swaplevel(0,1).unstack().copy()
output.columns = ["_".join(col) for col in output.columns]
output = combined_df.join(binding_df).fillna(0)
#output.to_csv("/nas3/gpratt/Dropbox/TAF15/Data/fold_change_and_binding.csv")

In [ ]:
#HTML(output[output['tdp43_distintron500'] > 0].head().to_html())

In [ ]:
#result = defaultdict(dict)
#for name, df in sig_df.groupby(level=0):
#    bound_df = df.ix[name].join(binding_df).fillna(0)
#    for column in binding_df.columns:
#        result[name][column] = bound_df[bound_df[column] > 0]['log2FoldChange'].values
#        if len(result[name][column]) == 0:
#            result[name][column] = [0]
#result = pd.DataFrame(result)
#result.to_csv("/nas3/gpratt/Dropbox/TAF15/Data/significant_values.csv")

In [ ]:
#with dataviz.Figure(os.path.join(fig_dir, "foo.svg"), figsize=(6,24)) as fig:
#    for x, rbp in enumerate(result.drop("tia1", axis=1).columns):
#        rbp_name = rbp.split("_")[0]
#        if rbp_name == "tls":
#            rbp_name = "fus"
#        ax = fig.add_subplot(4,1,x + 1)
#        ax.boxplot(result[rbp][[index for index in result[rbp].index if rbp_name in index]])
#        ax.set_xticklabels([index for index in result[rbp].index if rbp_name in index], rotation=90)
#        ax.set_ylim(-5, 8)

#For Anthony, getting Human Genes that are bound

http://uswest.ensembl.org%2C%20ip-10-249-130-98.us-west-1.compute.internal:8000/biomart/martview/5f7dcf4b81c7c351d1d1af7d51d0f5ba?VIRTUALSCHEMANAME=default&ATTRIBUTES=mmusculus_gene_ensembl.default.homologs.ensembl_gene_id|mmusculus_gene_ensembl.default.homologs.ensembl_transcript_id|mmusculus_gene_ensembl.default.homologs.hsapiens_homolog_ensembl_gene&FILTERS=mmusculus_gene_ensembl.default.filters.with_homolog_hsap.only&VISIBLEPANEL=attributepanel

In [ ]:
gene_id_to_name = region_helpers.gene_id_to_name("/nas3/gpratt/gencode/gencode.v17.annotation.gtf.db")
gene_id_to_name = {key.split(".")[0]:value for key, value in gene_id_to_name.items()}

In [ ]:
filtered_binding

In [ ]:
mouse_human_genes = pd.read_table("/nas3/gpratt/projects/taf15/mouse_human_genes.txt", index_col=0)
mouse_human_genes = dict(mouse_human_genes['Human Ensembl Gene ID'].iterkv())

filtered_binding = binding_df.ix[[index.split(".")[0] in mouse_human_genes for index in binding_df.index.get_level_values("gene_id")]]
filtered_binding['human_gene_id'] = [mouse_human_genes[index.split(".")[0]] for index in filtered_binding.index.get_level_values("gene_id")]

#filtered_binding_hg19 = filtered_binding.ix[[index.split(".")[0] in gene_id_to_name.values for index in filtered_binding.index]]
#filtered_binding_hg19.index = [gene_id_to_name[index] for index in filtered_binding.index if index in index in gene_id_to_name]

filtered_binding.to_csv("/nas3/gpratt/Dropbox/TAF15/Data/mRNA_stab/binding_mapping.csv")

#Motif Analysis

In [ ]:
plot_denovo_motifs(rbps['tdp43'])
plot_denovo_motifs(rbps['fus'])
plot_denovo_motifs(rbps['taf15'])

#GO Enrichment

In [ ]:
#Done in Supp figure 1

#Get spatial clustering of motifs

In [ ]:
!findMotif -motif=GGTAAGT -bedOutput /nas3/yeolab/Genome/ucsc/mm9/chromosomes/all.fa  > ggtaagt.bed

In [ ]:
assigned_regions, regions = CLIP_analysis.regions_generator()

In [ ]:

def intersection(a, b=None):
    
    """
    
    A : bedtool
    B : bedtool
    Returns A - B and A intersect B 
    A with b and returns everything in a but not b and everything in a but... ???
    
    """
    
    overlapping = a.intersect(b, wa=True, u=True, stream=True).saveas()
    remaining = a.intersect(b, wa=True, v=True, stream=True).saveas()

    return remaining, overlapping

In [ ]:
def fix_shuffled_strand(shuffled_tool, regions_file):
    """
    Fixes strand of shuffled tool if there was an include file used

    Chooses the first overlapping thing as the correct strand, if there is anti-sense transcription
    this will fail
    shuffled_tool: a sorted shuffled bedtool
    shuffled_file: incl file that did the shuffling
    """

    regions_tool = pybedtools.BedTool(regions_file)
    intersected = shuffled_tool.intersect(regions_file, wao=True, stream=True, sorted=True).saveas()

    #Don't think about refactoring this because of leaky files in pybedtools
    shuffled_tool_field_count = shuffled_tool.field_count()
    regions_tool_field_count = regions_tool.field_count()

    total_header_size = shuffled_tool_field_count + regions_tool_field_count + 1

    intersected = intersected.to_dataframe(names=range(total_header_size))
    intersected = intersected.groupby([0,1,2,3]).first()
    if regions_tool.file_type == "bed":
        intersected[5] = intersected[shuffled_tool_field_count + 5]
    if regions_tool.file_type == "gff":
        intersected[5] = intersected[shuffled_tool_field_count + 6]

    values = intersected.apply(lambda x: "\t".join([str(item) for item in list(x.name) + list(x[:shuffled_tool_field_count - 4])]), axis=1).values
    new_bedtool = pybedtools.BedTool("\n".join(values), from_string=True)
    return new_bedtool


In [ ]:
def assign_to_regions(tool, clusters=None, assigned_dir=".", species="hg19", nrand=3):
    
    """
    
    Assigns each cluster to a genic region
    finally saves all generated bed and fasta files for future analysis...

    tool - a bed tool (each line represnting a cluster)
    clusters - name of cluster file (optional)
    assigned_dir - location to save files in
    species - str species to segment
    nrand - int number offsets times to shuffle for null hypothesis


    """
    if clusters is None:
        clusters, ext = os.path.splitext(os.path.basename(tool.fn))
    bedtracks = {}

    regions, assigned_regions = CLIP_analysis.regions_generator()

    for region in regions:
        bedtracks[region] = pybedtools.BedTool(os.path.join(clipper.data_dir(), "regions", "%s_%s.bed" % (species,
                                                                                                          region)))
              
    #creates the basics of bed dict
    bed_dict = {'all': {'rand': {}}}

    remaining_clusters = tool

    # print "There are a total %d clusters I'll examine" % (len(tool))
    for region in regions:
        remaining_clusters, overlapping = intersection(remaining_clusters, b=bedtracks[region])

        #if for some reason there isn't a peak in the region skip it
        if len(overlapping) == 0:
            # print "ignoring %s " % region
            continue

        #sets up bed dict for this region
        bed_dict[region] = {'real': overlapping.sort(stream=True).saveas(),
                            'rand': {}}

        no_overlapping_count = len(remaining_clusters)
        overlapping_count = len(bed_dict[region]['real'])
        print "For region: %s found %d that overlap and %d that don't" % (region,
                                                                          overlapping_count,
                                                                          no_overlapping_count)

        if 'real' not in bed_dict['all']:
            bed_dict['all']['real'] = bed_dict[region]['real']
        else:
            bed_dict['all']['real'] = bed_dict['all']['real'].cat(bed_dict[region]['real'], stream=True, postmerge=False).saveas()

        #saves offsets so after shuffling the offsets can be readjusted
        for i in range(nrand):
            random_intervals = bed_dict[region]['real'].shuffle(genome=species, incl=bedtracks[region].fn).sort()
            random_intervals = fix_shuffled_strand(random_intervals, bedtracks[region].fn)
            bed_dict[region]['rand'][i] = random_intervals.saveas()

            if i not in bed_dict['all']['rand']:
                bed_dict['all']['rand'][i] = bed_dict[region]['rand'][i]
            else:
                bed_dict['all']['rand'][i] = bed_dict['all']['rand'][i].cat(bed_dict[region]['rand'][i], stream=True, postmerge=False)

        #if there are no more clusters to assign stop trying
        if no_overlapping_count == 0:
            break

    # print "After assigning %d un-categorized regions" % len(remaining_clusters)

    if len(remaining_clusters) > 0:
        bed_dict['uncatagorized'] = {'real': remaining_clusters.sort(stream=True).saveas()}

    bed_dict = save_bedtools(bed_dict, clusters, assigned_dir)
    return bed_dict

def save_bedtools(cluster_regions, clusters, assigned_dir):
    """
    Given cluster regions file saves all bedtools sanely and returns result
    :param cluster_regions:
    :return:
    """
    for region in cluster_regions:
        output_file = "%s.%s.real.BED" % (clusters, region)
        cluster_regions[region]['real'] = cluster_regions[region]['real'].sort().saveas(os.path.join(assigned_dir, output_file))

        if "rand" not in cluster_regions[region]:
            continue

        for n_rand in cluster_regions[region]['rand']:
            output_file = "%s.%s.rand.%s.BED" % (clusters, region, n_rand)
            cluster_regions[region]['rand'][n_rand] = cluster_regions[region]['rand'][n_rand].sort().saveas(os.path.join(assigned_dir, output_file))

    return cluster_regions

In [ ]:
genes = pybedtools.BedTool("/nas3/gpratt/clipper/clipper/data/regions/mm9_genes.bed")

rbps = {"ggaatg_motif": pybedtools.BedTool("ggtaagt.bed")}

for name, rbp in rbps.items():
    overlapping_genes = rbp.intersect(genes, wo=True, s=True).each(move_name).saveas()
    cluster_regions = assign_to_regions(overlapping_genes, "ggaatg_taf15_motif", "assigned", nrand=10, species="mm9")

In [ ]:
real = cluster_regions['all']['real']

In [ ]:
slopped_tool = real.slop(b=250, g="/nas3/yeolab/Genome/ucsc/mm9/mm9.chrom.sizes")
overlapping_count_df = slopped_tool.intersect(real, c=True, s=True).to_dataframe(names=['chrom', 
                                                                 'start', 
                                                                 'stop', 
                                                                 'name', 
                                                                 'score',
                                                                 'strand',
                                                                 "chrom2",
                                                                 'start2',
                                                                 'stop2',
                                                                 'name2',
                                                                 'score2',
                                                                 'strand2',
                                                                 'something',
                                                                 'num_overlapping_targets'])

In [ ]:
result = {}
for val, rand in cluster_regions['all']['rand'].items():
    slopped_tool = rand.slop(b=250, g="/nas3/yeolab/Genome/ucsc/mm9/mm9.chrom.sizes")

    result[val] = slopped_tool.intersect(rand, c=True, s=True).to_dataframe(names=['chrom', 
                                                                 'start', 
                                                                 'stop', 
                                                                 'name', 
                                                                 'score',
                                                                 'strand',
                                                                 "chrom2",
                                                                 'start2',
                                                                 'stop2',
                                                                 'name2',
                                                                 'score2',
                                                                 'strand2',
                                                                 'something',
                                                                 'num_overlapping_targets'])
    
result = pd.concat(result, names=['trial', 'motif_num'])

In [ ]:
real = (overlapping_count_df.num_overlapping_targets - 1).mean()

In [ ]:
real

In [ ]:
result.num_overlapping_targets = result.num_overlapping_targets - 1

In [ ]:
rand = result.groupby(level="trial").num_overlapping_targets.mean().mean()

In [ ]:
import scipy.stats

In [ ]:

scipy.stats.poisson.sf(real , rand)